In [1]:
import whisper
model = whisper.load_model("base")
audio = whisper.load_audio("/opt/ml/final_test_0127/dataset/test03.mp3")   # 1분 50초 동영상

/opt/conda/envs/lv3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
import pandas as pd
dataset = pd.read_csv('/opt/ml/final/pl/datamodule/iemocap_full_dataset.csv')

In [18]:
import whisper

/opt/conda/envs/whisper/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
emo = ['neu', 'hap', 'sad', 'ang', 'fea']
emo_selected = dataset.loc[dataset.emotion.isin(emo)]
audio_list = []
for path in emo_selected['path']:
        audio = whisper.load_audio(path)
        audio = whisper.pad_or_trim(audio)    
        mel = whisper.log_mel_spectrogram(audio)
        mel = mel[None, :, :]  
        audio_list.append(mel.tolist())
out_dataset = pd.DataFrame(
        {
            "audio" : audio_list,
            "label": dataset["label"],
        }
    )
out_dataset

KeyboardInterrupt: 

In [ ]:
out_dataset['audio'].dtype

In [29]:
tmp = []
import os
for p in dataset['path']:
    path = p.split('/')[-1]
    path = path.split('.')[0]
    tmp.append(path)

In [30]:
dataset['path'] = tmp

In [31]:
dataset.to_csv('/opt/ml/final/pl/datamodule/iemocap_full_dataset.csv')

In [2]:
audio   #(1779206,)

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [3]:
audio.shape

(2525184,)

In [4]:
# 오디오 입력의 샘플링 속도를 모델에서 예상하느 샘플링 속도와 일치시키기
# Whisper_feature_extractor는 2가지 작업을 수행해야함.
# 1-1. 모든 샘플의 입력길이가 30초가 되도록 오디오 샘플 배치를 채우거나 자르기(padding)
# 1-2. 1-1번에서 작업한 패딘된 오디오 array를 log-Mel spectrogram으로 변환

# 1-1. 30초로 분활되어 패딩 및 자르기
audio = whisper.pad_or_trim(audio)          
audio       # audio.len : (480000,)

array([0.        , 0.        , 0.        , ..., 0.07836914, 0.07250977,
       0.03768921], dtype=float32)

In [5]:
# 1-2. log-Mel spectrogram으로 변환 후 모델로 전달
mel = whisper.log_mel_spectrogram(audio).to(model.device)
mel     # torch.Size([80, 3000])

tensor([[-0.6634, -0.6634, -0.0352,  ...,  0.1270,  0.3081,  0.2873],
        [-0.6634, -0.6634, -0.0857,  ...,  0.0288,  0.2825,  0.2265],
        [-0.6634, -0.6634, -0.1781,  ...,  0.0614,  0.0889,  0.3709],
        ...,
        [-0.6634, -0.6546,  0.0634,  ...,  0.0476,  0.0112, -0.0835],
        [-0.6634, -0.6634, -0.1304,  ..., -0.0162,  0.0729, -0.0101],
        [-0.6634, -0.6634, -0.2362,  ..., -0.0752,  0.0377, -0.0628]],
       device='cuda:0')

In [6]:
mel.shape

torch.Size([80, 3000])

In [7]:
# 모델의 encoder에 mel tensor 입력하기

# dim 맞추기
mel = mel[None, :, :]       # torch.Size([1, 80, 3000])
encoder_output = model.embed_audio(mel)
encoder_output              # torch.Size([1, 1500, 512])    (batch_size, 1500(number of spectrogram), hidden_size)

tensor([[[-0.1422,  0.7148, -0.4809,  ...,  0.2073,  1.0649, -0.1069],
         [ 0.8354,  0.6639, -0.4328,  ..., -0.0761,  1.1189, -0.3803],
         [ 0.2717,  0.6384, -0.0670,  ..., -0.5671, -0.1905, -1.0390],
         ...,
         [ 0.5807, -0.5485, -2.2702,  ..., -1.6035, -0.6112, -0.4250],
         [ 0.2030, -0.2901, -1.1699,  ..., -0.9606, -0.5970, -1.1560],
         [-0.7661,  1.0489, -0.2621,  ..., -0.1690, -0.4695, -0.8974]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)

In [8]:
encoder_output.shape

torch.Size([1, 1500, 512])

---